In [2]:
import pandas as pd
import json
import os

# Configuracion
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

# Modelos en el mismo orden que aparecen en los CSVs
MODEL_NAMES = ["Amazon_Nova_Micro_V1", "Cohere_Command_R_08_2024"]

# Crear carpeta para datos procesados
OUTPUT_DIR = 'Data_for_Metrics'
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Carpeta: {OUTPUT_DIR}")

Carpeta: Data_for_Metrics


## 1. Cargar Dataset Original

In [3]:
# Cargar dataset original (ground truth)
dataset_path = '../modismos_Dataset_Final.csv'
df_dataset = pd.read_csv(dataset_path, delimiter=';')

# -------- Eliminar duplicados manteniendo primera ocurrencia !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

df_dataset_unique = df_dataset.drop_duplicates(subset=['modismo'], keep='first')
print(f"Dataset: {len(df_dataset_unique)} modismos unicos")

Dataset: 6531 modismos unicos


## 2. Funciones de Extracción

In [4]:
def extract_from_json(json_str, field):
    """Extrae un campo especifico del JSON de respuesta."""
    try:
        data = json.loads(json_str)
        output = data.get('output', {})
        return output.get(field, '')
    except:
        return ''

def extract_nested_input(json_str, field):
    """Extrae un campo del input cuando este es un objeto (Prompt 4)."""
    try:
        data = json.loads(json_str)
        input_data = data.get('input', {})
        if isinstance(input_data, dict):
            return input_data.get(field, '')
        return ''
    except:
        return ''

## 3. PROMPT 1: Modismo → Definición

In [5]:
print("Procesando Prompt 1...")

# Cargar respuestas
df_p1 = pd.read_csv('../Straico/responses_prompt_1.csv')

# Extraer definiciones de cada modelo
model_columns = [col for col in df_p1.columns if col != 'modismo']
for col in model_columns:
    df_p1[f'{col}_definicion'] = df_p1[col].apply(lambda x: extract_from_json(x, 'definicion'))

# Merge con dataset original
df_p1_merged = df_p1.merge(
    df_dataset_unique[['modismo', 'significado']], 
    on='modismo', 
    how='left'
)

# Crear DataFrame final usando MODEL_NAMES
df_p1_final = pd.DataFrame({
    'modismo': df_p1_merged['modismo'],
    'definicion_real': df_p1_merged['significado'],
    MODEL_NAMES[0]: df_p1_merged[f'{model_columns[0]}_definicion'],
    MODEL_NAMES[1]: df_p1_merged[f'{model_columns[1]}_definicion']
})

# Guardar
output_file = os.path.join(OUTPUT_DIR, 'prompt_1_metrics_data.csv')
df_p1_final.to_csv(output_file, index=False)
print(f"  Guardado: {output_file} ({df_p1_final.shape[0]} filas)")

Procesando Prompt 1...
  Guardado: Data_for_Metrics/prompt_1_metrics_data.csv (10 filas)


## 4. PROMPT 2: Modismo → Es Modismo (Sí/No)

In [6]:
print("Procesando Prompt 2...")

# Cargar respuestas
df_p2 = pd.read_csv('../Straico/responses_prompt_2.csv')

# Extraer respuestas de cada modelo
model_columns = [col for col in df_p2.columns if col != 'modismo']
for col in model_columns:
    df_p2[f'{col}_respuesta'] = df_p2[col].apply(lambda x: extract_from_json(x, 'es_modismo'))

# Crear DataFrame final usando MODEL_NAMES
df_p2_final = pd.DataFrame({
    'modismo': df_p2['modismo'],
    'es_modismo_real': 'Sí',  # Todos los del dataset son modismos
    MODEL_NAMES[0]: df_p2[f'{model_columns[0]}_respuesta'],
    MODEL_NAMES[1]: df_p2[f'{model_columns[1]}_respuesta']
})

# Guardar
output_file = os.path.join(OUTPUT_DIR, 'prompt_2_metrics_data.csv')
df_p2_final.to_csv(output_file, index=False)
print(f"  Guardado: {output_file} ({df_p2_final.shape[0]} filas)")

Procesando Prompt 2...
  Guardado: Data_for_Metrics/prompt_2_metrics_data.csv (10 filas)


## 5. PROMPT 3: Definición → Modismo

In [7]:
print("Procesando Prompt 3...")

# Cargar respuestas
df_p3 = pd.read_csv('../Straico/responses_prompt_3.csv')

# Extraer palabras/modismos generados por cada modelo
model_columns = [col for col in df_p3.columns if col not in ['significado', 'modismo_original']]
for col in model_columns:
    df_p3[f'{col}_palabra'] = df_p3[col].apply(lambda x: extract_from_json(x, 'palabra'))

# Crear DataFrame final usando MODEL_NAMES
df_p3_final = pd.DataFrame({
    'definicion': df_p3['significado'],
    'modismo_real': df_p3['modismo_original'],
    MODEL_NAMES[0]: df_p3[f'{model_columns[0]}_palabra'],
    MODEL_NAMES[1]: df_p3[f'{model_columns[1]}_palabra']
})

# Guardar
output_file = os.path.join(OUTPUT_DIR, 'prompt_3_metrics_data.csv')
df_p3_final.to_csv(output_file, index=False)
print(f"  Guardado: {output_file} ({df_p3_final.shape[0]} filas)")

Procesando Prompt 3...
  Guardado: Data_for_Metrics/prompt_3_metrics_data.csv (10 filas)


## 6. PROMPT 4: Modismo + Ejemplo → Literal (Sinónimo) + Definición

In [8]:
print("Procesando Prompt 4...")

# Cargar respuestas
df_p4 = pd.read_csv('../Straico/responses_prompt_4.csv')

# Extraer literal (sinonimo) y definicion de cada modelo
model_columns = [col for col in df_p4.columns if col not in ['modismo', 'ejemplo']]

for col in model_columns:
    df_p4[f'{col}_literal'] = df_p4[col].apply(lambda x: extract_from_json(x, 'literal'))
    df_p4[f'{col}_definicion'] = df_p4[col].apply(lambda x: extract_from_json(x, 'definicion'))

# Merge con dataset para obtener significado real
df_p4_merged = df_p4.merge(
    df_dataset_unique[['modismo', 'significado']], 
    on='modismo', 
    how='left'
)

# Funcion auxiliar para sustituir modismo por sinonimo en el ejemplo
def sustituir_modismo(ejemplo, modismo, sinonimo):
    if pd.isna(ejemplo) or pd.isna(modismo) or pd.isna(sinonimo):
        return ''
    return ejemplo.replace(modismo, sinonimo)

# CSV 1: Ejemplo completo con sustitucion del sinonimo
for i, model_col in enumerate(model_columns):
    df_p4_merged[f'{MODEL_NAMES[i]}_ejemplo_con_sinonimo'] = df_p4_merged.apply(
        lambda row: sustituir_modismo(row['ejemplo'], row['modismo'], row[f'{model_col}_literal']),
        axis=1
    )

df_p4_csv1 = pd.DataFrame({
    'modismo': df_p4_merged['modismo'],
    'ejemplo_real': df_p4_merged['ejemplo'],
    f'{MODEL_NAMES[0]}_sinonimo': df_p4_merged[f'{model_columns[0]}_literal'],
    f'{MODEL_NAMES[0]}_ejemplo_con_sinonimo': df_p4_merged[f'{MODEL_NAMES[0]}_ejemplo_con_sinonimo'],
    f'{MODEL_NAMES[1]}_sinonimo': df_p4_merged[f'{model_columns[1]}_literal'],
    f'{MODEL_NAMES[1]}_ejemplo_con_sinonimo': df_p4_merged[f'{MODEL_NAMES[1]}_ejemplo_con_sinonimo']
})

# CSV 2: Solo modismo y sinonimos
df_p4_csv2 = pd.DataFrame({
    'modismo': df_p4_merged['modismo'],
    f'{MODEL_NAMES[0]}_sinonimo': df_p4_merged[f'{model_columns[0]}_literal'],
    f'{MODEL_NAMES[1]}_sinonimo': df_p4_merged[f'{model_columns[1]}_literal']
})

# CSV 3: Comparacion de definiciones
df_p4_csv3 = pd.DataFrame({
    'modismo': df_p4_merged['modismo'],
    'definicion_real': df_p4_merged['significado'],
    f'{MODEL_NAMES[0]}_definicion_sinonimo': df_p4_merged[f'{model_columns[0]}_definicion'],
    f'{MODEL_NAMES[1]}_definicion_sinonimo': df_p4_merged[f'{model_columns[1]}_definicion']
})

# Guardar los 3 CSVs
output_file_1 = os.path.join(OUTPUT_DIR, 'prompt_4_ejemplos_sustitucion.csv')
output_file_2 = os.path.join(OUTPUT_DIR, 'prompt_4_sinonimos.csv')
output_file_3 = os.path.join(OUTPUT_DIR, 'prompt_4_definiciones.csv')

df_p4_csv1.to_csv(output_file_1, index=False)
df_p4_csv2.to_csv(output_file_2, index=False)
df_p4_csv3.to_csv(output_file_3, index=False)

print(f"  Guardado: {output_file_1} ({df_p4_csv1.shape[0]} filas)")
print(f"  Guardado: {output_file_2} ({df_p4_csv2.shape[0]} filas)")
print(f"  Guardado: {output_file_3} ({df_p4_csv3.shape[0]} filas)")

Procesando Prompt 4...
  Guardado: Data_for_Metrics/prompt_4_ejemplos_sustitucion.csv (10 filas)
  Guardado: Data_for_Metrics/prompt_4_sinonimos.csv (10 filas)
  Guardado: Data_for_Metrics/prompt_4_definiciones.csv (10 filas)


## 7. Resumen de Archivos Generados

In [9]:
print("\nResumen de archivos generados:")
print("-" * 60)

import glob
files = glob.glob(os.path.join(OUTPUT_DIR, '*.csv'))
files.sort()

for file in files:
    filename = os.path.basename(file)
    df_temp = pd.read_csv(file)
    print(f"{filename}: {df_temp.shape[0]} x {df_temp.shape[1]}")

print("-" * 60)
print(f"Total: {len(files)} archivos en {OUTPUT_DIR}/")
print("Procesamiento completado.")


Resumen de archivos generados:
------------------------------------------------------------
prompt_1_metrics_data.csv: 10 x 4
prompt_2_metrics_data.csv: 10 x 4
prompt_3_metrics_data.csv: 10 x 4
prompt_4_definiciones.csv: 10 x 4
prompt_4_ejemplos_sustitucion.csv: 10 x 6
prompt_4_sinonimos.csv: 10 x 3
------------------------------------------------------------
Total: 6 archivos en Data_for_Metrics/
Procesamiento completado.
